In [1]:
import os

%pwd # Print current directory

'/Users/wilsvenleong/Downloads/learning-materials/chicken-disease-classification/research'

In [2]:
os.chdir("../")  # Move up to root directory
%pwd

'/Users/wilsvenleong/Downloads/learning-materials/chicken-disease-classification'

## 1. Update `params.yaml` and `config.yaml`

## 4. Update the Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class CallbacksConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoints_file_path: Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    epochs: int
    batch_size: int
    augmentation: bool
    image_size: list

## 5. Update the `ConfigurationManager` in `src/config/configuration.py`

In [4]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_callbacks_config(self) -> CallbacksConfig:
        callbacks_preparation = self.config.callbacks_preparation

        create_directories(
            [callbacks_preparation.root_dir, callbacks_preparation.tensorboard_log_dir]
        )

        callbacks_config = CallbacksConfig(
            root_dir=Path(callbacks_preparation.root_dir),
            tensorboard_log_dir=Path(callbacks_preparation.tensorboard_log_dir),
            checkpoints_file_path=Path(callbacks_preparation.checkpoints_file_path),
        )

        return callbacks_config

    def get_model_training_config(self) -> TrainingConfig:
        model_training = self.config.model_training
        base_model_preparation = self.config.base_model_preparation
        params = self.params

        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir, "Chicken-fecal-images"
        )

        create_directories([model_training.root_dir])

        model_training_config = TrainingConfig(
            root_dir=Path(model_training.root_dir),
            trained_model_path=Path(model_training.trained_model_path),
            updated_base_model_path=Path(
                base_model_preparation.updated_base_model_path
            ),
            training_data=Path(training_data),
            epochs=params.EPOCHS,
            batch_size=params.BATCH_SIZE,
            augmentation=params.AUGMENTATION,
            image_size=params.IMAGE_SIZE,
        )

        return model_training_config

## 6. Update the Components

In [5]:
import time
import tensorflow as tf


class Callbacks:
    def __init__(self, config: CallbacksConfig):
        self.callbacks_config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.callbacks_config.tensorboard_log_dir, f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.callbacks_config.checkpoints_file_path),
            save_best_only=True,
        )

    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]


class Training:
    def __init__(self, config: TrainingConfig):
        self.model_training_config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.model_training_config.updated_base_model_path
        )

    def train_valid_generator(self):
        data_generator_kwargs = {"rescale": 1.0 / 255, "validation_split": 0.2}
        data_flow_kwargs = {
            "target_size": self.model_training_config.image_size[:-1],
            "batch_size": self.model_training_config.batch_size,
            "interpolation": "bilinear",
        }

        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory=self.model_training_config.training_data,
            subset="validation",
            shuffle=False,
            **data_flow_kwargs,
        )

        if self.model_training_config.augmentation:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **data_generator_kwargs,
            )
        else:
            train_data_generator = valid_data_generator

        self.train_generator = train_data_generator.flow_from_directory(
            directory=self.model_training_config.training_data,
            subset="training",
            shuffle=True,
            **data_flow_kwargs,
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = (
            self.train_generator.samples // self.train_generator.batch_size
        )
        self.validation_steps = (
            self.valid_generator.samples // self.valid_generator.batch_size
        )

        self.model.fit(
            self.train_generator,
            epochs=self.model_training_config.epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list,
        )

        self.save_model(
            path=self.model_training_config.trained_model_path, model=self.model
        )

## 7. Update the Pipeline

In [6]:
try:
    config_manager = ConfigurationManager()

    callbacks_config = config_manager.get_callbacks_config()
    callbacks = Callbacks(config=callbacks_config)
    callbacks_list = callbacks.get_tb_ckpt_callbacks()

    model_training_config = config_manager.get_model_training_config()
    trainer = Training(config=model_training_config)
    trainer.get_base_model()
    trainer.train_valid_generator()
    trainer.train(callback_list=callbacks_list)
except Exception as e:
    raise e

[ 2023-09-23 01:55:54,497 ] 33 common chicken-disease-logger -  INFO - Loaded YAML file successfully from: config/config.yaml
[ 2023-09-23 01:55:54,500 ] 33 common chicken-disease-logger -  INFO - Loaded YAML file successfully from: params.yaml
[ 2023-09-23 01:55:54,501 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts
[ 2023-09-23 01:55:54,502 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts/callbacks
[ 2023-09-23 01:55:54,503 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts/callbacks/tensorboard_logs
[ 2023-09-23 01:55:54,507 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts/model_training
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
19/19 [==============================] - 43s 2s/step - loss: 9.8186 - accuracy: 0.6250 - val_loss: 7.8278 - val_accuracy: 0.6094


/Users/wilsvenleong/opt/anaconda3/envs/chicken/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
